In [1]:
from wdtools import *

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
set_ID = 'Set002'

In [3]:
# this will take about 17 minutes
start = time.time()
all_taxlot = combine_taxlot()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 16.0 minutes to complete


In [38]:
df1 = combined_reindexed_data(setID='Set001', nm_to_add=0) 

In [39]:
res = check_corrected_data(df=df1, setID='Set001', all_taxlot=all_taxlot, nm_to_add=0, export=False)

In [40]:
df2 = combined_reindexed_data(setID='Set002', nm_to_add=1420) 

In [41]:
set2gdf = match_wd_data_with_taxlot(df=df2, setID=set_ID, all_taxlot=all_taxlot, nm_to_add=1420, export=False)

In [5]:
ndf = res[0]

In [6]:
ngdf = gpd.GeoDataFrame(ndf, crs="EPSG:2992", geometry='geometry')

In [7]:
ngdf.columns

Index(['wdID', 'trsqq', 'parcel_id', 'loc_desc', 'city', 'county', 'site_name',
       'site_desc', 'latitude', 'longitude', 'coord_src', 'doc_name',
       'doc_link', 'isbatfile', 'status_nm', 'receiveddt', 'responsedt',
       'reissuance', 'project_id', 'site_id', 'recordID', 'notes', 'lots',
       'lot', 'cnt_code', 'ORTaxlot', 'recyear', 'IDyear', 'missinglot',
       'record_ID', 'geometry'],
      dtype='object')

In [8]:
ngdf.head(3)

,wdID,trsqq,parcel_id,loc_desc,city,county,site_name,site_desc,latitude,longitude,...,notes,lots,lot,cnt_code,ORTaxlot,recyear,IDyear,missinglot,record_ID,geometry
0,WD2017-0003,06N10W03DA,500 (partial),McCormick Rd .025 mi S of Hillila Rd,Gearhart,Clatsop,McCormick Rd 06N10W03DA,Residential,46.030197,-123.905217,...,partial,500,500,4,0406.00N10.00W03DA--000000500,2017,2017,N,1,"MULTIPOLYGON (((448821.559 1578307.121, 448814..."
1,WD2017-0033,08N08W25,101,"W of 92260 Koppisch Rd, Astoria",Astoria,Clatsop,08N08W25,site evaluation,46.155778,-123.610751,...,None,101,101,4,0408.00N08.00W2500--000000101,2017,2017,N,2,"MULTIPOLYGON (((524028.003 1621190.023, 524220..."
2,WD2017-0056,08N10W09B,"2500, ROWs (both partial)",Warrenton Dr and NW 17th St,Warrenton,Clatsop,Gann Dev 08N10W09B,residential development,46.190638,-123.932177,...,ROW & partial,2500,2500,4,0408.00N10.00W09B0--000002500,2017,2017,N,3,"MULTIPOLYGON (((443198.754 1636916.627, 443123..."


In [10]:
ngdf['lots'] = ngdf['lots'].apply(lambda x: ' '.join(dict.fromkeys(x).keys()))

In [11]:
ngdf[['wdID', 'trsqq', 'parcel_id', 'notes', 'lots', 'lot', 'ORTaxlot', 'record_ID', 'geometry']].to_file(os.path.join(inpath + '\\output\\matched\\', f'matched_records_{set_ID}.shp'), 
             driver='ESRI Shapefile')  

In [12]:
df_wlots_to_check = res[3]

In [13]:
df_wlots_to_check.to_csv(os.path.join(inpath + '\\output\\to_review\\', 'df_with_lots_check.csv'), index=False) 

In [43]:
wd_df = combine_wd_tables(setID='Set002', nm_to_add=1420)

In [44]:
wd_df.shape

(723, 25)

In [45]:
len(wd_df.record_ID.unique()) - len(set2gdf.record_ID.unique())

130

In [16]:
len(ngdf.record_ID.unique())

1331

In [17]:
wd_df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [48]:
setgdf = gpd.read_file(os.path.join(inpath, 'GIS', 'Join_Statewide.gdb'), layer=f'WD_{set_ID}_Combined')

In [49]:
len(setgdf.Record_ID.unique())

618

In [46]:
setgdf.columns

Index(['OBJECTID', 'Record_ID', 'wetdet_delin_number', 'trsqq', 'parcel_id',
       'address_location_desc', 'city', 'county', 'site_name', 'site_desc',
       'latitude', 'longitude', 'Coord_Source', 'DocumentName', 'DecisionLink',
       'is_batch_file', 'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'lot_num', 'lot',
       'Match_Year', 'ORTAXLOT', 'Match_found', 'Manual_note', 'Shape_Length',
       'Shape_Area', 'geometry'],
      dtype='object')

In [50]:
# missed records in the existing output
wd_df.shape[0] - len(setgdf.Record_ID.unique())

105

In [51]:
unmatched_wd_df = report_unmatched(gdf=set2gdf, setID=set_ID, nm_to_add=1420, mute=False)

it is about 17.98% of data in the original 723 records unmatched
there are 1 records (0.14% of the original records) without parcel id
